<a href="https://colab.research.google.com/github/Indukurivigneshvarma/Deep_Learning/blob/main/NLP/GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers --quiet
from datasets import load_dataset


In [2]:
dataset = load_dataset("blended_skill_talk")

print(dataset)
print(dataset["train"][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 4819
    })
    validation: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 980
    })
})
{'personas': ["i've 2 kids.", 'i love flowers.'], 'additional_context': '', 'previous_utterance': ["I love live music, that's why I try to go to concerts", 'I do too. Wat do you like?'], 'context': 'empathetic_dialogues', 'free_messages': ['I like acting, I hope to 

In [3]:
def preprocess_function(example):
    dialogues = []
    for conv in example["previous_utterance"]:
        conversation = " ".join(conv)
        dialogues.append(conversation)
    return {"text": dialogues}

processed_dataset = dataset["train"].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print(processed_dataset)

Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 4819
})


In [4]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    result = tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = processed_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

In [5]:
from torch.utils.data import random_split

train_size = int(0.9 * len(tokenized_dataset))
val_size = len(tokenized_dataset) - train_size
train_dataset, val_dataset = random_split(tokenized_dataset, [train_size, val_size])


In [6]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [7]:
from transformers import Trainer, TrainingArguments
import os

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./gpt2-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [8]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,1.244800
100,0.932900
150,0.928200
200,0.846800
250,0.928200
300,0.899900
350,0.886900
400,1.008400
450,0.939900
500,0.930800


TrainOutput(global_step=2169, training_loss=0.9042227346262881, metrics={'train_runtime': 246.707, 'train_samples_per_second': 17.58, 'train_steps_per_second': 8.792, 'total_flos': 283305885696000.0, 'train_loss': 0.9042227346262881, 'epoch': 1.0})

In [10]:
model.to("cuda")

prompt = "User: Hello, how are you?\nBot:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_length=80,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_p=0.9,
    temperature=0.8,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

User: Hello, how are you?
Bot: I am in the process of getting a new job.


In [11]:
from IPython.display import clear_output
clear_output(wait=False)
